In [2]:
from multiprocessing import Process, Queue, Pool
import os
from stupid_tails import run_experiment
from markov import markov_chain, markov_iter

In [3]:
def run_experiment_multi(n, M, q):

    exp = dict()

    exp["M"] = M
    exp["n"] = n

    parse = set("")

    tail_symbols = ""

    for k in range(1, n + 1):
        seq = ""
        m_iter = markov_iter(M)

        while seq in parse:
            seq += next(m_iter)

        parse.add(seq)
        tail_symbols += next(m_iter)

        # print("\tThis tree was {} characters long,\n\twith tail symbol {}".format(len(seq), tail_symbols[-1]))

        exp[str(k)] = seq

    exp["tail_symbols"] = tail_symbols
    
    q.put(exp)

In [17]:
%%time

M = markov_chain(2)
n = 100

exps = [run_experiment(n, M) for _ in range(200)]

CPU times: user 753 ms, sys: 575 ms, total: 1.33 s
Wall time: 814 ms


In [1]:
%%timeit -n10 -r3

from multiprocessing import Pool

def run_simu():
    pass
        
pool = Pool()
[pool.apply_async(run_simu) for _ in "1"]


TypeError: 'Pool' object is not iterable

In [15]:
%%time

M = markov_chain(2)
n = 10
n_exp = 10
jobs = []
q = Queue()

def run_simu(n, M, N, q):
    for _ in range(N):
        run_experiment_multi(n, M, q)

N = n_exp // 4
pool = Pool()
print([pool.apply_async(run_simu, (n, M, N, q)) for _ in "1234"])

OSError: [Errno 24] Too many open files

In [ ]:
keep = True
i = 0
while keep:
    try:
        q.get()
        i += 1
    except:
        keep = False
print(i)
    